In [1]:
import nest_asyncio
nest_asyncio.apply()
import pydra

### Introduction to Tasks with States

Task might be run for a single set of input values or we can generate multiple sets, that will be called "states". In order to create `Task` wit a `State` we have to provide input that is iterable and specify the way how we want to map values of the inputs to the specific states. In order to do it, we use the `split` method and specify `splitter`, for a function with one input, the `splitter` is trivial:

In [9]:
@pydra.to_task
def add_two(x):
    return x + 2

task1 = add_two(name="add_two", x=[1, 2]).split(splitter="x")
task1(plugin="cf")

We can now check the results of our task:

In [10]:
task1.result()

[Result(output=Output(out=3), runtime=None, errored=False),
 Result(output=Output(out=4), runtime=None, errored=False)]

As we could expect, the result now contains two elements, one for each value of `State`, i.e. for each value of `x` in this exampel.

Now, we can try to run function with multiple inputs:

In [8]:
@pydra.to_task
def add_var(a, b):
    return a + b

Now we have more options to use splitter, it depends on the input and on our application. We could have `a` that is a single value and `b` that is a list:

In [14]:
task2 = add_var(name="addvar1", a=100, b=[1,2]).split(splitter="b")
task2(plugin="cf")
task2.result()

[Result(output=Output(out=101), runtime=None, errored=False),
 Result(output=Output(out=102), runtime=None, errored=False)]

As in the previous example, our result contains two elements.

Now we can imagine that both, `a` and `b` are a two elements lists.

In [15]:
task3 = add_var(name="addvar1", a=[10, 100], b=[1, 2])

Now, we have two options to map the values, we might want to run the task for two sets of values: (`a`=10, `b`=1) and (`a`=100, `b`=2), or we might want to run the task for four sets: (`a`=10, `b`=1), (`a`=10, `b`=2), (`a`=100, `b`=1) and (`a`=100, `b`=2). 

The first situation will be represented by "scalar" splitter, the later by the "outer" splitter. 

Let's start from the scalar splitter, that uses parentheses in syntax:

In [16]:
task3.split(splitter=("a", "b"))
task3(plugin="cf")
task3.result()

[Result(output=Output(out=11), runtime=None, errored=False),
 Result(output=Output(out=102), runtime=None, errored=False)]

As we expected, we have `10+1=11` and `100+2=102`. 

For the outer splitter we will use brackets:

In [18]:
task4 = add_var(name="addvar1", a=[10, 100], b=[1, 2])
task4.split(splitter=["a", "b"])
task4(plugin="cf")
task4.result()

[Result(output=Output(out=11), runtime=None, errored=False),
 Result(output=Output(out=12), runtime=None, errored=False),
 Result(output=Output(out=101), runtime=None, errored=False),
 Result(output=Output(out=102), runtime=None, errored=False)]

Now, we have results for all of the combination of values of `a` and `b`.

For more inputs we can create more complex splitter by using scalar and outer products. Note, that the scalar splitter can only work for lists that have the same length, but the outer splitter doesn't have this limitation. Let's run one more example:

In [21]:
@pydra.to_task
def add_vector(x1, y1, x2, y2):
    return (x1 + x2, y1 + y2)

task5 = add_vector(name="add_vect", output_names=["x", "y"], 
                   x1=[10, 20], y1=[1, 2], x2=[10, 20, 30], y2=[10, 20, 30])
task5.split(splitter=[("x1", "y1"), ("x2", "y2")])
task5(plugin="cf")
task5.result()

[Result(output=Output(x=20, y=11), runtime=None, errored=False),
 Result(output=Output(x=30, y=21), runtime=None, errored=False),
 Result(output=Output(x=40, y=31), runtime=None, errored=False),
 Result(output=Output(x=30, y=12), runtime=None, errored=False),
 Result(output=Output(x=40, y=22), runtime=None, errored=False),
 Result(output=Output(x=50, y=32), runtime=None, errored=False)]

#### combining teh output

We could also combine specific outputs after using the `split` method. In order to do it, we can use `combine` method.

Let's say we want to calculate squares and cubes of integers and combine separately all squareas and all cubes: 

In [29]:
@pydra.to_task
def power(x, n):
    return x**n

task6 = power(name="power", x=[1, 2, 3, 4], n=[2, 3]).split(splitter=["x", "n"]).combine("x")
task6(plugin="cf")
task6.result()

[[Result(output=Output(out=1), runtime=None, errored=False),
  Result(output=Output(out=4), runtime=None, errored=False),
  Result(output=Output(out=9), runtime=None, errored=False),
  Result(output=Output(out=16), runtime=None, errored=False)],
 [Result(output=Output(out=1), runtime=None, errored=False),
  Result(output=Output(out=8), runtime=None, errored=False),
  Result(output=Output(out=27), runtime=None, errored=False),
  Result(output=Output(out=64), runtime=None, errored=False)]]

Now, our result contains two list, the first one is for squares, the second for cubes.

In [31]:
squares_list = [el.output.out for el in task6.result()[0]]
cubes_list = [el.output.out for el in task6.result()[1]]
print(f"squares: {squares_list}")
print(f"cubes: {cubes_list}")

squares: [1, 4, 9, 16]
cubes: [1, 8, 27, 64]
